In [2]:
import jcopdl, luwiji
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Dataset dan DataLoader

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

bs = 128
crop_size = 64

train_transform = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(crop_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(70),
    transforms.CenterCrop(crop_size),
    transforms.ToTensor()
])

train_set = datasets.ImageFolder("../../data CNN/train/", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=4)

test_set = datasets.ImageFolder("../../data CNN/test/", transform=test_transform)
testloader = DataLoader(test_set, batch_size=bs, shuffle=True, num_workers=4)

In [5]:
features, labels = next(iter(trainloader))
features.shape

torch.Size([128, 3, 64, 64])

In [6]:
label2cat = train_set.classes
label2cat

['invasive', 'noninvasive']

# Arsitektur & Config

In [ ]:
# nn.Conv2d(in_channels= 3, out_channels= 8, kernel_size= 3, stride=1, padding=1)
# nn.ReLU()
# nn.MaxPool2d(kernel_size=2, stride=2)

In [ ]:
from jcopdl.layers import conv_block, linear_block

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # c_out itu filter
        self.conv = nn.Sequential(
          # awalnya gambarnya 64 x 64
            conv_block(c_in= 3, c_out= 8, pool_kernel='max'), # 32 x 32
            conv_block(c_in= 8, c_out= 16, pool_kernel='max'), # 16 x 16
            conv_block(c_in= 16, c_out= 32, pool_kernel='max'), # 8 x 8
            conv_block(c_in= 32, c_out= 64, pool_kernel='max'), # 4 x 4
            nn.Flatten(),
        )
        
        self.fc = nn.Sequential(
          # nah abis kena pooling, trus 64 nya itu dia dari c_out conv terakhir
          linear_block(64*4*4, 256, activation='relu', dropout=0.1),
          linear_block(256, 2, activation="lsoftmax")
        )
        
        
    def forward(self, x):
      self.conv(x)
      self.fc(x)
      return x
